<a href="https://colab.research.google.com/github/eunicean/Proyecto-DeepLearning/blob/main/CNN/drawing_hiragana_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
!git clone https://github.com/eunicean/Proyecto-DeepLearning.git

fatal: destination path 'Proyecto-DeepLearning' already exists and is not an empty directory.


In [27]:
# pa pintar
!pip install ipycanvas pillow -q

In [33]:
from google.colab import output
output.enable_custom_widget_manager()

In [29]:
# Imports y configuración
import tensorflow as tf
from tensorflow import keras
import numpy as np
from PIL import Image
from ipycanvas import Canvas, hold_canvas
from ipywidgets import Button, HBox, VBox, Output, Layout
from IPython.display import display, clear_output
import matplotlib.pyplot as plt
import random

# Configuración
IMG_HEIGHT = 64
IMG_WIDTH = 64
IMG_CHANNELS = 1

# Cargar modelo
MODEL_PATH = '/content/Proyecto-DeepLearning/models/best_cnn_hiragana_model.h5'
model = keras.models.load_model(MODEL_PATH)
print("Modelo cargado")

# Definir clases (orden alfabético - mismo que entrenamiento)
class_labels = ['aa', 'chi', 'ee', 'fu', 'ha', 'he', 'hi', 'ho', 'ii',
                'ka', 'ke', 'ki', 'ko', 'ku', 'ma', 'me', 'mi', 'mo',
                'mu', 'na', 'ne', 'ni', 'nn', 'no', 'nu', 'oo', 'ra',
                're', 'ri', 'ro', 'ru', 'sa', 'se', 'shi', 'so', 'su',
                'ta', 'te', 'tsu', 'to', 'uu', 'wa', 'wo', 'ya', 'yo', 'yu']

Modelo cargado


In [34]:
# Celda 2: Canvas y variables
canvas_size = 400
visual_size = 300

canvas = Canvas(
    width=canvas_size,
    height=canvas_size,
    sync_image_data=True,
    layout=Layout(width=f'{visual_size}px', height=f'{visual_size}px')
)

canvas.fill_style = 'white'
canvas.fill_rect(0, 0, canvas_size, canvas_size)
canvas.stroke_style = 'black'
canvas.line_width = 15

drawing = False
last_x = 0
last_y = 0
output = Output()

RANDOM_OBJECTIVE_LETTER = random.choice(class_labels)
print(f"LETRA A DIBUJAR: {RANDOM_OBJECTIVE_LETTER}")

# Celda 3: Funciones de dibujo
def on_mouse_down(x, y):
    global drawing, last_x, last_y
    drawing = True
    last_x = x
    last_y = y

def on_mouse_move(x, y):
    global drawing, last_x, last_y
    if drawing:
        with hold_canvas(canvas):
            canvas.stroke_style = 'black'
            canvas.line_width = 15
            canvas.begin_path()
            canvas.move_to(last_x, last_y)
            canvas.line_to(x, y)
            canvas.stroke()
            last_x = x
            last_y = y

def on_mouse_up(x, y):
    global drawing
    drawing = False

canvas.on_mouse_down(on_mouse_down)
canvas.on_mouse_move(on_mouse_move)
canvas.on_mouse_up(on_mouse_up)

# Celda 4: Funciones de control
def change_letter(button):
    global RANDOM_OBJECTIVE_LETTER
    RANDOM_OBJECTIVE_LETTER = random.choice(class_labels)
    canvas.clear()
    canvas.fill_style = 'white'
    canvas.fill_rect(0, 0, canvas_size, canvas_size)
    with output:
        clear_output()
        print(f"Nueva letra: {RANDOM_OBJECTIVE_LETTER}")

def clear_canvas(button):
    global RANDOM_OBJECTIVE_LETTER
    canvas.clear()
    canvas.fill_style = 'white'
    canvas.fill_rect(0, 0, canvas_size, canvas_size)
    with output:
        clear_output()
        print(f"Canvas limpiado - Letra actual: {RANDOM_OBJECTIVE_LETTER}")

def predict_drawing(button):
    global RANDOM_OBJECTIVE_LETTER
    with output:
        clear_output()
        print(f"Letra objetivo: {RANDOM_OBJECTIVE_LETTER}")
        print("Analizando...\n")

        try:
            img_data = canvas.get_image_data()
            img_array = np.array(img_data, dtype=np.uint8)

            if img_array.size == 0:
                print("No hay dibujo")
                return

            img_gray = img_array[:, :, 0]
            img_pil = Image.fromarray(img_gray, mode='L')
            img_resized = img_pil.resize((64, 64), Image.Resampling.LANCZOS)
            img_normalized = np.array(img_resized) / 255.0
            img_input = img_normalized.reshape(1, 64, 64, 1)

            prediction = model.predict(img_input, verbose=0)

            predicted_idx = np.argmax(prediction[0])
            predicted_class = class_labels[predicted_idx]
            confidence = prediction[0][predicted_idx] * 100

            top_n = 10
            top_indices = np.argsort(prediction[0])[-top_n:][::-1]

            print(f"PREDICCION: {predicted_class}")
            print(f"CONFIANZA: {confidence:.2f}%")

            # la idea podria ser usar la predicción para darle una nota al usuario
            # ahora mismo el modelo es muy estricto con los dibujos
            # entonces podemos aprovechar eso para poder puntuar el dibujo
            # por ejemplo, si el usuario hace un dibujo muy bueno y la predicción 1
            # del modelo concuerda con lo que se le pidio dibujar al usuario tiene 10/10
            # si el usuario dibuja y el modelo predice como 4to candidato la clase solicitada
            # entonces tiene 6/10 puntos
            # para una nota sobre 10, se puede hacer una formula como 10 - x + 1
            # donde x es la posicion en la que quedó la clase objetivo en la predicción
            # por ejemplo
            # si se pide dibujar 'ka' y luego de que el usuario haga el dibujo
            # el modelo predice ka en la 4 posición, entonces la nota es
            # 10 - (4) + 1 = 7

            points = 0
            print(f"\nTop {top_n}:")
            for i, idx in enumerate(top_indices, 1):
                class_name = class_labels[idx]
                prob = prediction[0][idx] * 100
                print(f" {i}. {class_name:5s} : {prob:6.2f}%")

                if class_name == RANDOM_OBJECTIVE_LETTER:
                    points = 10 - i + 1

            print(f"\nPuntuacion: {points}/10")

        except Exception as e:
            print(f"Error: {e}")
            import traceback
            traceback.print_exc()

btn_clear = Button(description='Limpiar', button_style='warning')
btn_predict = Button(description='Predecir', button_style='success')
btn_cambiar = Button(description='Cambiar', button_style='info')

btn_clear.on_click(clear_canvas)
btn_predict.on_click(predict_drawing)
btn_cambiar.on_click(change_letter)

display(VBox([
    canvas,
    HBox([btn_clear, btn_predict, btn_cambiar]),
    output
]))

LETRA A DIBUJAR: no
